In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import html.parser
import requests
import json
import csv
import time

In [64]:
f_data = ["Product Link","TITLE","DESCTRIPTION","SKU (CODE)","SIZES","AVAILABILITY at my City","Images","Price €","Category > Subcategory > Subcategory > Subcategory"]
with open('output_90.csv', 'w', newline='', encoding="utf-8", errors="ignore") as new_file:
    csv_writer = csv.writer(new_file)
    csv_writer.writerow(f_data)

In [65]:
urls = []
with open('urls_90.csv', 'r' ) as csv_file:
    csv_reader = csv.reader(csv_file)
    for line in csv_reader:
        url = str(line[0]).strip()
        urls.append(url)
print(len(urls))

489


In [66]:
headers = {
    'authority': 'www.e-jumbo.gr',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': url,
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

params = (
    ('lang', 'el'),
)

In [ ]:
driver = webdriver.Chrome()

In [53]:
url = "https://www.e-jumbo.gr/eidi-pet-shop/volta-skylou/samarakia-skylou/samaraki-anapaftiko-bez-kafe-small_1349348/"
driver.get(url)

In [67]:
for i in range(0,len(urls)):
    url = urls[i]
    driver.get(url)

    versions = driver.find_elements_by_css_selector('[class="versions-container"] [class*="x-product-details-version-selector"] li')
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        name = str(soup.select_one('h1[itemprop="name"]').text).strip()
    except:
        continue

    try:
        price = str(soup.select_one('[class="current-price"]').text).strip()
    except:
        price = ""

    try:    
        sku  = str(soup.select_one('[class="code"]').text).replace("Κωδικός Jumbo:","").strip()
    except:
        sku  = ""

    try:
        category = ""
        cats = soup.select('[class="breadcrumb clearfix"] [class="title"]')
        for cat in cats[:4]:
            category += str(cat.text).strip() + " > "
        category = category.strip(" > ")
    except:
        category = ""

    try:
        images = ""
        images_raw=soup.select('a[class="cloud-zoom"]')
        for img in images_raw:
            images += "https:" + str(img["href"]).strip().split(".jpg?")[0] + ".jpg" + "\n"
        images = images.strip("\n") 
        #print(images)
    except:
        images = ""

    try:
        description = ""
        sections = soup.select('[class="tabs-content"] [class*="tab-box"]')
        for sec in sections:
            label = str(sec.select_one('[class="title"]').text).strip()
            if label =="Περιγραφή - Χαρακτηριστικά":
                description = str(sec.select_one('[class="dynamic-content"]').text).strip()
    except:
        description = ""

    f_data = [url,name,description,"","","",images,price,category]

    with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(f_data)    

    print(i)

    if len(versions)==0:

        sku   = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skucode"]).strip()
        skuId = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skuid"]).strip()
        #print(sku,skuId)

        data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
        data = json.dumps(data)
        #print(data)

        r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
        w_data = json.loads(r_2.json()["d"]["data"])

        availability = "Μη διαθέσιμο"

        try:
            warehouses   = w_data["WarehousesGrouped"][0]["Items"]
        except:
            warehouses   =  []
            availability = str(soup.select_one('[class*="availability-text"]').text).strip()

        for wh in warehouses:
            area = str(wh["Area"]).strip()
            if area == "Λάρνακα":
                availability = str(wh["AvailabilityName"]).strip()

        f_data = ["","","",sku,"None",availability,"","",""]

        with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
            csv_writer = csv.writer(new_file)
            csv_writer.writerow(f_data)

        print(i,r_2.status_code)

    for j in range(len(versions)):
        versions = driver.find_elements_by_css_selector('[class="versions-container"] [class*="x-product-details-version-selector"] li')
        size  = str(versions[j].find_element_by_css_selector('[class="size"]').text).strip()
        versions[j].click()
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        
        sku   = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skucode"]).strip()
        skuId = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skuid"]).strip()

        #print(sku,skuId)

        data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
        data = json.dumps(data)
        #print(data)

        r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
        w_data = json.loads(r_2.json()["d"]["data"])

        availability = "Μη διαθέσιμο"

        try:
            warehouses   = w_data["WarehousesGrouped"][0]["Items"]
        except:
            warehouses   =  []
            availability = str(soup.select_one('[class*="availability-text"]').text).strip()

        for wh in warehouses:
            area = str(wh["Area"]).strip()
            if area == "Λάρνακα":
                availability = str(wh["AvailabilityName"]).strip()

        f_data = ["","","",sku,size,availability,"","",""]

        with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
            csv_writer = csv.writer(new_file)
            csv_writer.writerow(f_data)

        print(i,r_2.status_code)

0
0 200
1
1 200
2
2 200
3
3 200
4
4 200
5
5 200
6
6 200
7
7 200
8
8 200
9
9 200
10
10 200
11
11 200
12
12 200
13
13 200
14
14 200
15
15 200
16
16 200
17
17 200
18
18 200
19
19 200
20
20 200
21
21 200
22
22 200
23
23 200
24
24 200
25
25 200
26
26 200
27
27 200
28
28 200
29
29 200
30
30 200
31
31 200
32
32 200
33
33 200
34
34 200
35
35 200
36
36 200
37
37 200
38
38 200
39
39 200
40
40 200
41
41 200
42
42 200
43
43 200
44
44 200
45
45 200
46
46 200
47
47 200
48
48 200
49
49 200
50
50 200
51
51 200
52
52 200
53
53 200
54
54 200
55
55 200
56
56 200
57
57 200
58
58 200
59
59 200
60
60 200
61
61 200
62
62 200
63
63 200
64
64 200
65
65 200
66
66 200
67
67 200
68
68 200
69
69 200
70
70 200
71
71 200
72
72 200
73
73 200
74
74 200
75
75 200
76
76 200
77
77 200
78
78 200
79
79 200
80
80 200
81
81 200
82
82 200
83
83 200
84
84 200
85
85 200
86
86 200
87
87 200
88
88 200
89
89 200
90
90 200
91
91 200
92
92 200
93
93 200
94
94 200
95
95 200
96
96 200
97
97 200
98
98 200
99
99 200
100
100 200
101
101 

In [68]:
import pandas as pd
import numpy as np

In [69]:
# Reading the csv file
df_new = pd.read_csv('output_90.csv')
  
# saving xlsx file
GFG = pd.ExcelWriter('output_90.xlsx')
df_new.to_excel(GFG, index = False)
  
GFG.save()

In [70]:
GFG.close()